In [1]:
import torch
import warnings
warnings.filterwarnings('ignore')
from sklearn.cluster import SpectralClustering
from sklearn.cluster import KMeans
from torch_geometric.utils import to_dense_adj
from torch_geometric.utils import to_networkx
from torch_geometric.datasets import Planetoid, WebKB, WikipediaNetwork,Actor,LINKXDataset
import networkx as nx
import numpy as np
np.random.seed(1234)
def spectral_cluster(graph, num_clusters):
    normalized_laplacian_matrix = nx.normalized_laplacian_matrix(graph).todense()
    eigvals, eigvecs = np.linalg.eig(normalized_laplacian_matrix)
    eigvals = np.real(eigvals)
    eigvecs = np.real(eigvecs)
    #print(eigvals)
    #print(eigvecs)
    indices = np.argsort(eigvals)[:num_clusters]
    k_smallest_eigvecs = eigvecs[:, indices]
    clustering = KMeans(n_clusters=num_clusters).fit(k_smallest_eigvecs)
    cluster_assignments = clustering.labels_
    return list(cluster_assignments)

def structural_heterophily(data,num_classes=5):
    G = to_networkx(data, to_undirected=True)    
    # Get the largest connected component
    Gcc = sorted(nx.connected_components(G), key=len, reverse=True)
    G = G.subgraph(Gcc[0])
    # data.y will contain the labels only of the largest connected component
    selected_nodes = list(G.nodes)
    data.y = data.y[selected_nodes]
    laplacian = nx.normalized_laplacian_matrix(G)
    laplacian = laplacian.todense()
    # to tensor
    laplacian = torch.tensor(laplacian,dtype=torch.float32)
    data.y = torch.tensor(data.y.numpy(),dtype=torch.float32)
    # Let's compute data.y^T @ L @ data.y
    num = torch.matmul(torch.matmul(data.y,laplacian),data.y)
    # Let's compute sc.labels_^T @ L @ sc.labels_
    sc = spectral_cluster(G,num_classes)
    sc = torch.tensor(sc,dtype=torch.float32)
    den = torch.matmul(torch.matmul(sc,laplacian),sc)
    return round((num/den).item(),3)
dataset = WebKB(root='./', name='Cornell')
#dataset = Planetoid(root='/tmp/Cora', name='Pubmed')
#dataset = WikipediaNetwork(root='./',name = 'Squirrel')
#dataset = Actor(root='./')

print(dataset)
print(dataset[0])
data = dataset[0]

print(structural_heterophily(data,num_classes=data.y.max().item()+1))

Penn94(1)
Data(x=[41554, 4814], edge_index=[2, 2724458], y=[41554], train_mask=[41554, 5], val_mask=[41554, 5], test_mask=[41554, 5])


# Table with the results of the structure heterophily experiment
|  | Cora | Citeseer | Pubmed | Actor | Chamaleon | Squirrel | Wisconsin  | Cornell | Texas | Penn94 |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| SH |7.367 | 5.779 | 7.642 |209.586 | 8.307 | 20.629 | 6.908 |6.038 | 18.376 | 00.000 |